In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
!pip install ultralytics -q

In [4]:
import os
HOME = os.getcwd()
print(HOME)

/kaggle/working


In [5]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics 8.3.43 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
Setup complete ✅ (4 CPUs, 31.4 GB RAM, 6013.5/8062.4 GB disk)


In [6]:
import random
from PIL import Image
import cv2

from ultralytics import YOLO

from IPython.display import display, Image

In [7]:
label_mapping = {
"aegypti": 0,
"albopictus": 1,
"anopheles": 2,
"culex": 3,
"culiseta": 4,
"japonicus/koreicus": 5
}

In [27]:
labels = {value: key for key, value in label_mapping.items()}
labels

{0: 'aegypti',
 1: 'albopictus',
 2: 'anopheles',
 3: 'culex',
 4: 'culiseta',
 5: 'japonicus/koreicus'}

# Testing an image

In [23]:
model = YOLO(f'{HOME}/yolov5s.pt')
results = model.predict(source='https://media.roboflow.com/notebooks/examples/dog.jpeg', conf=0.25, imgsz=640)

PRO TIP 💡 Replace 'model=/kaggle/working/yolov5s.pt' with new 'model=/kaggle/working/yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.


Found https://media.roboflow.com/notebooks/examples/dog.jpeg locally at dog.jpeg
image 1/1 /kaggle/working/dog.jpeg: 640x384 1 person, 1 car, 1 dog, 1 backpack, 25.0ms
Speed: 2.7ms preprocess, 25.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 384)


In [34]:
results[0].orig_img.shape

(1280, 720, 3)

In [10]:
results[0].boxes.xyxy

tensor([[6.4177e+01, 2.4365e+02, 6.4631e+02, 1.2641e+03],
        [6.2999e+02, 7.3656e+02, 6.9717e+02, 7.8825e+02],
        [7.6184e-01, 4.3309e+02, 4.6251e+02, 1.2772e+03],
        [0.0000e+00, 3.6454e+02, 6.5941e+02, 1.2734e+03]], device='cuda:0')

In [11]:
results[0].boxes.conf

tensor([0.6859, 0.4160, 0.3914, 0.3310], device='cuda:0')

## Format Dataset

[Dataset Format](https://docs.ultralytics.com/datasets/detect/#ultralytics-yolo-format)

In [ ]:
ds_dir="/kaggle/input"

In [ ]:
cd /kaggle/working

In [27]:
rm -rf images/train images/val labels/train labels/val 

In [28]:
mkdir -p images/train images/val labels/train labels/val

In [29]:
cp -r /kaggle/input/dlp-object-detection/final_dlp_data/final_dlp_data/train/images/* ./images/train

In [30]:
cp -r /kaggle/input/dlp-object-detection/final_dlp_data/final_dlp_data/train/labels/* ./labels/train

In [ ]:
ds_yaml="""
# Train/val/test sets as 1) dir: path/to/imgs, 2) file: path/to/imgs.txt, or 3) list: [path/to/imgs1, path/to/imgs2, ..]
path: /kaggle/working/ # dataset root dir
train: images/train # train images (relative to 'path') 4 images
val: images/val # val images (relative to 'path') 4 images
test: # test images (optional)

# Classes (Mosquito classes)
names:
    0: aegypti
    1: albopictus
    2: anopheles
    3: culex
    4: culiseta
    5: "japonicus/koreicus"
    
"""

with open("ds.yaml", "w") as f:
    f.write(ds_yaml)

In [ ]:
cat ds.yaml

In [40]:
# class_statistics = {}
# labels_folder = "./labels/train"
# # Iterate through all text files in the labels folder
# for filename in os.listdir(labels_folder):
#     if filename.endswith('.txt'):
#         file_path = os.path.join(labels_folder, filename)
        
#         # Read the contents of the label file
#         with open(file_path, 'r') as file:
#             for line in file:
#                 # Split the line into fields
#                 fields = line.strip().split()
#                 if fields:
#                     label = fields[0]  # The first field is the label
#                     # Add the file path to the corresponding class in the dictionary
#                     if label not in class_statistics:
#                         class_statistics[label] = []
#                     class_statistics[label].append(file_path.split("/")[-1])

In [42]:
# import random
# import shutil
# # # Define paths
# train_images_dir = 'images/train'
# train_labels_dir = 'labels/train'
# val_images_dir = 'images/val'
# val_labels_dir = 'labels/val'

# total = 0
# for x in class_statistics.keys():
#     image_files = class_statistics[x]
#     val_size = int(len(image_files)*0.1)
#     val_images = random.sample(image_files, val_size)
#     for image in val_images:
#     # # Construct full file paths
#         image_path = os.path.join(train_images_dir, image.replace('.txt', '.jpeg'))
#         label_path = os.path.join(train_labels_dir, image)
#         # print(image_path, label_path, image)
#         # # # Move image to validation directory
#         shutil.move(image_path, os.path.join(val_images_dir, image.replace(".txt", ".jpeg")))
        
#         # # # # Move corresponding label to validation directory if it exists
#         if os.path.exists(label_path):
#             shutil.move(label_path, os.path.join(val_labels_dir, image))
#     total+=val_size

# print(f"Moved {total} images and their labels to the validation dataset.")

Moved 748 images and their labels to the validation dataset.


## Load and Train the Model

In [54]:
# Train the model
# the model on sumission is trained for 5 epochs and then trainind for 5 epochs on best.pt
model_path=f'{HOME}/yolo11s.pt'
model = YOLO(model_path)

In [55]:
results = model.train(data="ds.yaml", epochs=10, imgsz=640)

Ultralytics 8.3.43 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: task=detect, mode=train, model=/kaggle/working/yolo11s.pt, data=ds.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_b

train: Scanning /kaggle/working/labels/train.cache... 6752 images, 0 backgrounds, 1 corrupt: 100%|██████████| 6752/6752 [00:00<?, ?it/s]

train: WARNING ⚠️ /kaggle/working/images/train/120b30b0-c7db-4f0a-bead-a30424a65453.jpeg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0068]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /kaggle/working/labels/val.cache... 748 images, 0 backgrounds, 0 corrupt: 100%|██████████| 748/748 [00:00<?, ?it/s]


Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 4 dataloader workers
Logging results to runs/detect/train4
Starting training for 20 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      20/20      4.93G      0.841     0.6538      1.084         15        640: 100%|██████████| 422/422 [02:30<00:00,  2.80it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:11<00:00,  2.11it/s]


                   all        748        748      0.717      0.431      0.433      0.338

20 epochs completed in 0.928 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 19.1MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 19.1MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics 8.3.43 🚀 Python-3.10.14 torch-2.4.0 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLO11s summary (fused): 238 layers, 9,415,122 parameters, 0 gradients, 21.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:11<00:00,  2.08it/s]


                   all        748        748      0.715      0.457      0.454      0.355
               aegypti          3          3          1          0     0.0352     0.0336
            albopictus        333        333      0.823      0.922      0.925      0.706
             anopheles          5          5          1          0     0.0346     0.0321
                 culex        331        331      0.788      0.932      0.924      0.716
              culiseta         46         46      0.368      0.652      0.561      0.452
    japonicus/koreicus         30         30      0.311      0.233      0.247      0.193
Speed: 0.2ms preprocess, 3.7ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/train4


In [9]:
# rm -rf /kaggle/working/runs/detect

## Test the model

[Refer for test](https://github.com/roboflow/notebooks/blob/main/notebooks/train-yolov8-object-detection-on-custom-dataset.ipynb)

In [56]:
import os
import ast
best_model = YOLO("runs/detect/train4/weights/best.pt")
# test_set_loc="/kaggle/input/dlp-object-detection/final_dlp_data/final_dlp_data/test/images/"
# results = best_model.predict(source=test_set_loc, conf=0.1, imgsz=640)

In [57]:
def process_bboxes(results, image_id, image_name, image_height, image_width):
    detections = []
    print("ID:", image_id)
    
    for result in results:
        boxes = result.boxes.cpu().numpy()  # Extract bounding boxes
        
        if len(boxes) == 0:  # If no detections, generate a random label and values
            random_cls = random.randint(0, 5)  # Random class from dict1
            random_label = labels[random_cls]
            xmid = random.random()
            ymid = random.random()
            width = random.random()
            height = random.random()
            
            # Add random detection
            detections.append({
                'id': image_id,
                'ImageID': image_name,
                'LabelName': random_label,
                'Conf': random.random(),  # Random confidence
                'xcenter': xmid,
                'ycenter': ymid,
                'bbx_width': width,
                'bbx_height': height
            })
        else:
            # Find the box with the maximum confidence score
            max_conf_idx = np.argmax([box.conf[0] for box in boxes])
            box = boxes[max_conf_idx]  # Select the best box
            
            # Extract bounding box coordinates and class
            xyxy = box.xyxy[0]
            conf = box.conf[0]
            cls = int(box.cls[0])  # Convert to integer for indexing
            
            # Convert (xmin, ymin, xmax, ymax) to normalized (xmid, ymid, width, height)
            xmin, ymin, xmax, ymax = xyxy
            xmid = (xmin + xmax) / 2
            ymid = (ymin + ymax) / 2
            width = xmax - xmin
            height = ymax - ymin
            
            # Normalize bounding box to the image size
            xmid /= image_width
            ymid /= image_height
            width /= image_width
            height /= image_height
            
            # Add the best result to the list
            detections.append({
                'id': image_id,
                'ImageID': image_name,
                'LabelName': labels[cls],
                'Conf': np.round(conf, 3),
                'xcenter': np.round(xmid, 3),
                'ycenter': np.round(ymid, 3),
                'bbx_width': np.round(width, 3),
                'bbx_height': np.round(height, 3)
            })
    
    return detections

In [58]:
# Initialize a list to store all detections
detections = []

# Load all jpeg images from the folder
images = os.listdir("/kaggle/input/dlp-object-detection/final_dlp_data/final_dlp_data/test/images/")

for image_id, image_path in enumerate(sorted(images)):
    if not image_path.endswith('.jpeg'):
        continue  # Skip non-JPEG files
    
    print(f"Processing {image_path}")
    
    # Load the image and convert it to a numpy array
    image = Image.open("/kaggle/input/dlp-object-detection/final_dlp_data/final_dlp_data/test/images/" + image_path)
    image = np.array(image)  # Convert PIL image to numpy array
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  # Convert to BGR format for OpenCV
    
    # Perform inference
    results = best_model.predict(image)
    
    # Process bounding boxes and collect detections, passing image height and width
    image_height, image_width = image.shape[:2]
    detection = process_bboxes(results, image_id, image_path.split('.')[0], image_height, image_width)
    
    # Add the detections to the list
    detections.extend(detection)

Processing 0031063e-716a-4080-934c-77598dc8de72.jpeg

0: 640x480 1 culex, 11.0ms
Speed: 3.2ms preprocess, 11.0ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)
ID: 0
Processing 00fbfad7-9722-4581-831c-79faa576ea7f.jpeg

0: 640x480 (no detections), 9.3ms
Speed: 3.7ms preprocess, 9.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 480)
ID: 1
Processing 02043b0e-3d7d-4ca4-a36f-4bf97c344264.jpeg

0: 640x384 1 culex, 12.9ms
Speed: 2.6ms preprocess, 12.9ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 384)
ID: 2
Processing 0365513c-8f00-44f3-abd0-1fadde81c602.jpeg

0: 640x480 1 culex, 9.4ms
Speed: 2.9ms preprocess, 9.4ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 480)
ID: 3
Processing 0365d78b-2064-4d54-b15c-994d1950479a.jpeg

0: 640x512 1 albopictus, 11.2ms
Speed: 3.7ms preprocess, 11.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 512)
ID: 4
Processing 03d6749a-c5b3-45a3-81fa-b9c2f09eb4ba.jpeg

0: 576x640 1 albo

In [59]:
# Convert detections list to a DataFrame
df = pd.DataFrame(detections)

# Save to CSV
df.to_csv("submission.csv", index=False)
print("Detection results saved to submission.csv")

Detection results saved to submission.csv


In [60]:
ls -lsh

total 54M
104K -rw-r--r-- 1 root root 104K Dec  6 07:50 dog.jpeg
4.0K -rw-r--r-- 1 root root  468 Dec  6 07:50 ds.yaml
4.0K drwxr-xr-x 4 root root 4.0K Dec  6 07:51 images/
4.0K drwxr-xr-x 4 root root 4.0K Dec  6 07:51 labels/
4.0K drwxr-xr-x 3 root root 4.0K Dec  6 08:55 runs/
 52K -rw-r--r-- 1 root root  49K Dec  6 11:17 submission.csv
5.4M -rw-r--r-- 1 root root 5.4M Dec  6 07:51 yolo11n.pt
 19M -rw-r--r-- 1 root root  19M Dec  6 07:44 yolo11s.pt
5.4M -rw-r--r-- 1 root root 5.4M Dec  6 07:51 yolov5nu.pt
 18M -rw-r--r-- 1 root root  18M Dec  6 07:44 yolov5su.pt
6.3M -rw-r--r-- 1 root root 6.3M Dec  6 07:51 yolov8n.pt
